# Census Dissemination Areas


## Overview

Let's explore the data with [csvkit](http://csvkit.readthedocs.io/en/0.9.1/)

### List all (470) column names

And save it to [columns.txt](columns.txt)

```
csvcut -n census_dissemination.csv > columns.txt
```

### Look at the first 3 columns of the first 4 rows

In [1]:
%%bash
head -n 5 census_dissemination.csv | csvcut -c 1,2,3 | csvlook


|------------------------------------+------------------+--------------------------|
|  Geography                         | Population, 2011 | Total private dwellings  |
|------------------------------------+------------------+--------------------------|
|  Waterloo (3530)   00000           | 507096.00        | 202121.00                |
|  North Dumfries (3530004) TP 00000 | 9334.00          | 3288.00                  |
|  35300798 (35300798)   00000       | 587.00           | 204.00                   |
|  35300799 (35300799)   00000       | 376.00           | 135.00                   |
|------------------------------------+------------------+--------------------------|


### Find Geography values starting with a letter

In [2]:
%%bash
csvcut -c 1,2 census_dissemination.csv | csvgrep -c 1 -r "^[A-Z]" | csvlook

|------------------------------------+-------------------|
|  Geography                         | Population, 2011  |
|------------------------------------+-------------------|
|  Waterloo (3530)   00000           | 507096.00         |
|  North Dumfries (3530004) TP 00000 | 9334.00           |
|  Cambridge (3530010) CY 00000      | 126748.00         |
|  Kitchener (3530013) CY 00000      | 219153.00         |
|  Waterloo (3530016) CY 00000       | 98780.00          |
|  Wilmot (3530020) TP 00000         | 19223.00          |
|  Wellesley (3530027) TP 00000      | 10713.00          |
|  Woolwich (3530035) TP 00000       | 23145.00          |
|------------------------------------+-------------------|


### Population

The column values for "Population, 2011" and "Total population by age groups" don't quite match:

In [3]:
%%bash
head -n 10 census_dissemination.csv | csvcut -c 1,2,6 | csvlook


|------------------------------------+------------------+---------------------------------|
|  Geography                         | Population, 2011 | Total population by age groups  |
|------------------------------------+------------------+---------------------------------|
|  Waterloo (3530)   00000           | 507096.00        | 507095.00                       |
|  North Dumfries (3530004) TP 00000 | 9334.00          | 9335.00                         |
|  35300798 (35300798)   00000       | 587.00           | 585.00                          |
|  35300799 (35300799)   00000       | 376.00           | 375.00                          |
|  35300800 (35300800)   00000       | 439.00           | 440.00                          |
|  35300801 (35300801)   00000       | 450.00           | 450.00                          |
|  35300802 (35300802)   00000       | 452.00           | 455.00                          |
|  35300803 (35300803)   00000       | 418.00           | 420.00                

### Create a new file with line numbers added

In [12]:
%%bash
csvcut -l census_dissemination.csv > census_indexed.csv
head -n 5 census_indexed.csv | csvcut -c 1,2,3 | csvlook                                         


|--------------+-----------------------------------+-------------------|
|  line_number | Geography                         | Population, 2011  |
|--------------+-----------------------------------+-------------------|
|  1           | Waterloo (3530)   00000           | 507096.00         |
|  2           | North Dumfries (3530004) TP 00000 | 9334.00           |
|  3           | 35300798 (35300798)   00000       | 587.00            |
|  4           | 35300799 (35300799)   00000       | 376.00            |
|--------------+-----------------------------------+-------------------|


### Create new files with groups of related columns

In [13]:
%%bash
# dwellings and land area
csvcut -c 1,2,3,4,5,6 census_indexed.csv > census_dwellings.csv
csvcut -n census_dwellings.csv


  1: line_number
  2: Geography
  3: Population, 2011
  4: Total private dwellings
  5: Private dwellings occupied by usual residents
  6: Land area in square kilometres


In [14]:
%%bash
# age groups
csvcut -c 1,2,3,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32 census_indexed.csv > census_age_groups.csv
csvcut -n census_age_groups.csv


  1: line_number
  2: Geography
  3: Population, 2011
  4: Total population by age groups
  5:   0 to 4 years
  6:   5 to 9 years
  7:   10 to 14 years
  8:   15 to 19 years
  9:     15 years
 10:     16 years
 11:     17 years
 12:     18 years
 13:     19 years
 14:   20 to 24 years
 15:   25 to 29 years
 16:   30 to 34 years
 17:   35 to 39 years
 18:   40 to 44 years
 19:   45 to 49 years
 20:   50 to 54 years
 21:   55 to 59 years
 22:   60 to 64 years
 23:   65 to 69 years
 24:   70 to 74 years
 25:   75 to 79 years
 26:   80 to 84 years
 27:   85 years and over
 28: Median age of the population
 29: % of the population aged 15 and over


### Joining data files

In [7]:
%%bash
csvjoin -c "line_number" census_age_groups.csv census_dwellings.csv | csvgrep -c 2 -r "^[A-Z]" | csvcut -c 1,2,5,35 > census_join_example.csv
csvlook census_join_example.csv


|--------------+-----------------------------------+----------------+---------------------------------|
|  line_number | Geography                         |   0 to 4 years | Land area in square kilometres  |
|--------------+-----------------------------------+----------------+---------------------------------|
|  1           | Waterloo (3530)   00000           | 30940.00       | 1368.94                         |
|  2           | North Dumfries (3530004) TP 00000 | 445.00         | 187.44                          |
|  20          | Cambridge (3530010) CY 00000      | 7955.00        | 113.00                          |
|  210         | Kitchener (3530013) CY 00000      | 13705.00       | 136.79                          |
|  524         | Waterloo (3530016) CY 00000       | 4990.00        | 64.02                           |
|  678         | Wilmot (3530020) TP 00000         | 1165.00        | 263.72                          |
|  708         | Wellesley (3530027) TP 00000      | 985.00     

## Agate

In [8]:
import agate

# we can specify new column names for census_join_example.csv
column_names = ["id", "municipality", "population", "area"]

# load the csv into agate with the new column names
preschoolers = agate.Table.from_csv("census_join_example.csv", column_names)

preschoolers.print_table()

|------+----------------------+------------+-----------|
|   id | municipality         | population |     area  |
|------+----------------------+------------+-----------|
|    1 | Waterloo (3530)  ... |     30,940 | 1,368.94  |
|    2 | North Dumfries (3... |        445 |   187.44  |
|   20 | Cambridge (353001... |      7,955 |   113.00  |
|  210 | Kitchener (353001... |     13,705 |   136.79  |
|  524 | Waterloo (3530016... |      4,990 |    64.02  |
|  678 | Wilmot (3530020) ... |      1,165 |   263.72  |
|  708 | Wellesley (353002... |        985 |   277.79  |
|  728 | Woolwich (3530035... |      1,695 |   326.17  |
|------+----------------------+------------+-----------|


In [9]:
# we can refer to column values by name (row["id"]) or index (row[0])
for row in preschoolers.rows:
    municipality = row["municipality"].split(" (")[0] # extract just the name
    density = int(row["population"]/row["area"])
    print("{} has {} preschoolers per square kilometre".format(municipality, density))

Waterloo has 22 preschoolers per square kilometre
North Dumfries has 2 preschoolers per square kilometre
Cambridge has 70 preschoolers per square kilometre
Kitchener has 100 preschoolers per square kilometre
Waterloo has 77 preschoolers per square kilometre
Wilmot has 4 preschoolers per square kilometre
Wellesley has 3 preschoolers per square kilometre
Woolwich has 5 preschoolers per square kilometre


### Group Dissemination Areas by Municipality

Lines 3-19 appear to be dissemination areas for North Dumfries.

In [78]:
%%bash
head -n 21 census_indexed.csv | csvcut -c 1,2 > census_geography_example.csv
csvlook census_geography_example.csv


|--------------+------------------------------------|
|  line_number | Geography                          |
|--------------+------------------------------------|
|  1           | Waterloo (3530)   00000            |
|  2           | North Dumfries (3530004) TP 00000  |
|  3           | 35300798 (35300798)   00000        |
|  4           | 35300799 (35300799)   00000        |
|  5           | 35300800 (35300800)   00000        |
|  6           | 35300801 (35300801)   00000        |
|  7           | 35300802 (35300802)   00000        |
|  8           | 35300803 (35300803)   00000        |
|  9           | 35300804 (35300804)   00000        |
|  10          | 35300805 (35300805)   00000        |
|  11          | 35300806 (35300806)   00000        |
|  12          | 35300807 (35300807)   00000        |
|  13          | 35300808 (35300808)   00000        |
|  14          | 35300809 (35300809)   00000        |
|  15          | 35300810 (35300810)   00000        |
|  16          | 35300811 (3

In [21]:
%%bash
csvcut -c 1,2 census_indexed.csv | csvgrep -c 2 -r "^[A-Z]" > census_municipalities.csv
csvlook census_municipalities.csv

|--------------+------------------------------------|
|  line_number | Geography                          |
|--------------+------------------------------------|
|  1           | Waterloo (3530)   00000            |
|  2           | North Dumfries (3530004) TP 00000  |
|  20          | Cambridge (3530010) CY 00000       |
|  210         | Kitchener (3530013) CY 00000       |
|  524         | Waterloo (3530016) CY 00000        |
|  678         | Wilmot (3530020) TP 00000          |
|  708         | Wellesley (3530027) TP 00000       |
|  728         | Woolwich (3530035) TP 00000        |
|--------------+------------------------------------|


In [140]:
%%bash
csvcut -c 1,2 census_indexed.csv > census_areas_raw.csv
head -n 10 census_areas_raw.csv | csvlook

|--------------+------------------------------------|
|  line_number | Geography                          |
|--------------+------------------------------------|
|  1           | Waterloo (3530)   00000            |
|  2           | North Dumfries (3530004) TP 00000  |
|  3           | 35300798 (35300798)   00000        |
|  4           | 35300799 (35300799)   00000        |
|  5           | 35300800 (35300800)   00000        |
|  6           | 35300801 (35300801)   00000        |
|  7           | 35300802 (35300802)   00000        |
|  8           | 35300803 (35300803)   00000        |
|  9           | 35300804 (35300804)   00000        |
|--------------+------------------------------------|


In [137]:
# helper functions
def match_area_id(text):
    """
    Extract area/name and identifier from Census Geography column
    
    match_area_id("Waterloo (3530)   00000") == ("Waterloo", "3530")
    """
    
    import re
    _match = re.match("(.*) [(](.*)[)] .*", text)
    return _match.groups()

def get_ranges(start_values, all_values):
    """
    Use list of start_values to create ranges of items to extract from all_values
    
    get_ranges([1,3], [1,2,3,4,5]) == [(1, 2), (3, 5)]
    """

    stop_values = [value - 1 for value in start_values[1:]]
    stop_values.append(len(all_values))
    
    return zip(start_values, stop_values)

# load data
municipalities = agate.Table.from_csv("census_municipalities.csv")

areas = agate.Table.from_csv("census_areas_raw.csv")

# first row of municipalities table is Waterloo Region
region_row = municipalities.rows[0]
region_id = match_area_id(region_row["Geography"])

start_values = [int(row["line_number"]) for row in municipalities.rows[1:]] # skip first row

ranges = get_ranges(start_values, areas.rows)
names = [match_area_id(row["Geography"]) for row in municipalities.rows[1:]]
named_ranges = zip(names, ranges)

column_values = []
for n, r in named_ranges:
    start, stop = r
    for row in areas.rows[start:stop]:
        _, _id = match_area_id(row["Geography"])
        columns = list(region_id)
        columns.extend(list(n))
        columns.append(_id)
        column_values.append(columns)

column_names = ["Region", "region_id", "Municipality", "municipality_id", "area_id"]

areas_table = agate.Table(column_values, column_names)
areas_table.to_csv("census_areas.csv")

In [139]:
%%bash
head -n 5 census_areas.csv | csvlook

|-----------+-----------+----------------+-----------------+-----------|
|  Region   | region_id | Municipality   | municipality_id | area_id   |
|-----------+-----------+----------------+-----------------+-----------|
|  Waterloo | 3530      | North Dumfries | 3530004         | 35300798  |
|  Waterloo | 3530      | North Dumfries | 3530004         | 35300799  |
|  Waterloo | 3530      | North Dumfries | 3530004         | 35300800  |
|  Waterloo | 3530      | North Dumfries | 3530004         | 35300801  |
|-----------+-----------+----------------+-----------------+-----------|
